In [1]:
from env.dealers import SimpleDealer
from env.table import Table

from player.agents import SimpleAgent
from player.modules import *
from player.brains import SimpleBrain, NeuralHistoryCompressor

from utils import ConditionalExpectation_Loss

In [2]:
num_players = 9
num_agents = 12
num_cards = 52 + 1
bins = 11
actions_dim = bins + 4
embedding_hidden_dim = 7
feedforward_dim = 24
mlp_hidden = 36
embedding_output_dim_table = 2 + 5 * embedding_hidden_dim + actions_dim
embedding_output_dim_now = 3 + 7 * embedding_hidden_dim
history_compressor_depth = 100
start_credits = 1000

memory_params = [
    {
        "dim_source" : embedding_output_dim_table + actions_dim, 
        "dim_target" : embedding_output_dim_table,
        "transformer_params": {
            "d_model": actions_dim, 
            "nhead": 5, 
            "num_encoder_layers": 2, 
            "num_decoder_layers": 2, 
            "dim_feedforward": feedforward_dim
        }
    },

    {
        "dim_source" : embedding_output_dim_table + actions_dim, 
        "dim_target" : actions_dim,
        "transformer_params": {
            "d_model": actions_dim, 
            "nhead": 5, 
            "num_encoder_layers": 2, 
            "num_decoder_layers": 2, 
            "dim_feedforward": feedforward_dim
        }
    },
]

extractor_parameters = {
    "d_model": embedding_output_dim_table, 
    "nhead": 4, 
    "num_encoder_layers": 4, 
    "num_decoder_layers": 4, 
    "dim_feedforward": feedforward_dim
}

functions = [Fourier, Fourier, Fourier]
function_args = [
    {"dim_in": embedding_hidden_dim * 3, "depth": 15},
    {"dim_in": embedding_hidden_dim, "depth": 15},
    {"dim_in": embedding_hidden_dim, "depth": 15},
]

agent_args = {
    "start_type": TransformersInput,
    "start_args" : {
        "first_args" : {
            "dim_source" : embedding_output_dim_table, 
            "dim_target" : embedding_output_dim_now,
            "transformer_params": {
                "d_model": embedding_output_dim_table, 
                "nhead": 4, 
                "num_encoder_layers": 4, 
                "num_decoder_layers": 4, 
                "dim_feedforward": feedforward_dim
            }
        }, 
        "second_args" : {
            "dim_source" : embedding_output_dim_table + actions_dim, 
            "dim_target" : embedding_output_dim_table,
            "transformer_params": {
                "d_model": embedding_output_dim_table + actions_dim + 1, 
                "nhead": 4, 
                "num_encoder_layers": 4, 
                "num_decoder_layers": 4, 
                "dim_feedforward": feedforward_dim
            }
        }
    },

    "module_args" : {
        "First_Numbered_MLP" : {
            "dim_in" : actions_dim + embedding_output_dim_table * 2 + 1, 
            "hidden_dim" : mlp_hidden,
            "dim_out" : actions_dim, 
            "depth" : 3, 
            "num_agents": num_agents
        },
        
        "FourierOutput" : {
            "dim_in" : actions_dim, 
            "dim_encoder_output": actions_dim + embedding_output_dim_table * 2 + 1,
            "hidden_dim" : mlp_hidden,
            "shapes" : [[embedding_hidden_dim * 3, 15], [embedding_hidden_dim, 15], [embedding_hidden_dim, 15]], 
            "depth" : 3,
            "num_agents": num_agents
        }
    },

    "modules" : {
        "First_Numbered_MLP" : Numbered_MLP,
        "FourierOutput" : FourierOutput
    }
}

In [3]:
table = Table(num_players, bins = bins)

loss = ConditionalExpectation_Loss(start_credits, function_args, functions)
memory = NeuralHistoryCompressor(num_agents, history_compressor_depth, memory_params, extractor_parameters, train_freq = 20)
embedding = SimpleEmbedding(num_cards, embedding_hidden_dim, num_players, start_credits)

brain = SimpleBrain(num_agents, loss, memory, SimpleAgent, embedding, agent_args)

In [4]:
dealer = SimpleDealer(table, brain)

In [5]:
n_games = 10

dealer.game(n_games, num_players, 180, 3)

game  0  is finished!
game  1  is finished!
optimization end with time:  27.763246774673462
game  2  is finished!
game  3  is finished!
game  4  is finished!
optimization end with time:  4.72729754447937
game  5  is finished!
game  6  is finished!
game  7  is finished!
optimization end with time:  4.370973587036133
game  8  is finished!
game  9  is finished!
